## Building an Intelligent Airbnb Finder with MCP and Mistral Large

In this notebook, we'll build a personalized Airbnb recommendation system using Mistral Large with the Model Context Protocol (MCP). Our system will consider user preferences to provide tailored Airbnb recommendations.

We'll use the Airbnb MCP Server to search for listings, get details about properties, and understand locations.

Our system works as follows:

**User Request**: User asks for Airbnb recommendations with location data

**Location Analysis**: Airbnb MCP server processes location information

**Preference Consideration**: System considers user preferences for accommodations

**Listing Search**: Airbnb MCP searches for listings matching preferences

**Detail Enrichment**: Get specific details about promising Airbnb options

**Recommendation**: Format personalized recommendations based on user preferences

## Setting Up MCP Server

To set up our MCP server, we need the following installations:

### Install MCP client library
```
pip install mcp
```

### Install Airbnb MCP server
```
npm install -g @openbnb/mcp-server-airbnb
```

In [ ]:
!pip install -q mcp boto3 nest-asyncio

In [1]:

import boto3
import json
from typing import List, Dict, Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from contextlib import AsyncExitStack
import asyncio
import nest_asyncio
nest_asyncio.apply()
import time
import random

## The AirbnbFinder class handles:

**Server Connections**: Establishes connections to the Airbnb MCP server

**Tool Routing**: Routes tool calls to the appropriate MCP server

**Query Processing**: Manages the conversation with Mistral Large, handling tool requests

Our notebook initializes the server using the MCP client library:

In [ ]:
class AirbnbFinder:
    def __init__(self):
        self.exit_stack = AsyncExitStack()
        self.sessions = {}
        
    async def connect_to_mcp_servers(self):
        """Initialize connections to MCP servers"""
        
        servers_config = {
            "airbnb": StdioServerParameters(
                command="npx",
                args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"] #To ignore robots.txt for all requests
            )
        }
        
        for name, params in servers_config.items():
            try:
                print(f"Connecting to {name} server...")
                stdio_transport = await self.exit_stack.enter_async_context(
                    stdio_client(params)
                )
                stdio, write = stdio_transport
                session = await self.exit_stack.enter_async_context(
                    ClientSession(stdio, write)
                )
                self.sessions[name] = session
                await session.initialize()
                
                # Test the connection by getting available tools
                tools = await session.list_tools()
                print(f"Successfully connected to {name} server. Available tools:")
                for tool in tools.tools:  # Access the tools attribute
                    print(f"  - {tool.name}: {tool.description}")
                
            except Exception as e:
                print(f"Error connecting to {name} server: {str(e)}")
                print("Stack trace:")
                import traceback
                traceback.print_exc()
                raise

    async def get_available_tools(self) -> List[Dict]:
        """Get list of available tools from all MCP servers"""
        tools = []
        for server_name, session in self.sessions.items():
            try:
                response = await session.list_tools()
                tools.extend([{
                    "name": t.name,
                    "description": t.description,
                    "input_schema": t.inputSchema
                } for t in response.tools])  # Access response.tools
            except Exception as e:
                print(f"Error getting tools from {server_name}: {str(e)}")
        return tools

    async def execute_tool(self, tool_name: str, arguments: Dict) -> str:
       
        server_name = "airbnb"
        if server_name and server_name in self.sessions:
            try:
                print(f"Executing {tool_name} on {server_name} server")
                result = await self.sessions[server_name].call_tool(tool_name, arguments)
                return result.content
            except Exception as e:
                print(f"Error executing {tool_name} on {server_name} server: {str(e)}")
                raise ValueError(f"Error executing {tool_name}: {str(e)}")
        
        if not server_name:
            for name, session in self.sessions.items():
                try:
                    print(f"Trying {tool_name} on {name} server")
                    result = await session.call_tool(tool_name, arguments)
                    return result.content
                except Exception as e:
                    print(f"Failed on {name} server: {str(e)}")
                    continue
        
        raise ValueError(f"Tool {tool_name} not found in any MCP server")

    async def process_query(self, query: str, location: str = None):
        """Process a query using Mistral with tool access"""
        try:
            # Initialize conversation with just the user query
            user_message = f"Find Airbnb listings"
            if location:
                user_message += f" in {location}"
            user_message += f". {query}"
            
            conversation = [{
                "role": "user",
                "content": [{"text": user_message}]
            }]
            
            # Set up tools
            raw_tools = await self.get_available_tools()
            print("Available tools:", raw_tools)
            formatted_tools = []
            for tool in raw_tools:
                # print("###########TOOL#############",tool)
                formatted_tools.append({
                    "toolSpec": {
                        "name": tool["name"],
                        "description": tool["description"],
                        "inputSchema": {
                            "json": tool["input_schema"]
                        }
                    }
                })
            
            # Format system correctly
            system = [{"text": SYSTEM_PROMPT}]
            
            
            MAX_RETRIES = 5
            BASE_DELAY = 1  
            MAX_DELAY = 60
            
            bedrock = boto3.client('bedrock-runtime')
            MISTRAL_MODEL = 'mistral.mistral-large-2402-v1:0'
            
            while True:
                retry_count = 0
                while retry_count <= MAX_RETRIES:
                    try:
                        print(f"Sending request to Bedrock with {len(conversation)} messages in conversation")
                        print(f"Attempt {retry_count + 1}/{MAX_RETRIES + 1}")
                        
                        # Make API call
                        response = bedrock.converse(
                            modelId=MISTRAL_MODEL,
                            messages=conversation,
                            system=system,
                            toolConfig={
                                "tools": formatted_tools,
                                "toolChoice": {"auto": {}}
                            }
                        )
                        
                        # Get response data
                        output = response.get("output", {})
                        message = output.get("message", {})
                        content = message.get("content", [])
                        stop_reason = response.get("stopReason", "none")
                        
                        print(f"Got response with stop reason: {stop_reason}")
                        
                        # If we get here, the request succeeded, so break the retry loop
                        break
                        
                    except Exception as e:
                        retry_count += 1
                        if retry_count > MAX_RETRIES:
                            print(f"Max retries ({MAX_RETRIES}) exceeded. Last error: {e}")
                            raise
                        
                        # Calculate delay with exponential backoff and jitter
                        delay = min(BASE_DELAY * (2 ** (retry_count - 1)), MAX_DELAY)
                        
                        jitter = delay * 0.2
                        actual_delay = delay + random.uniform(-jitter, jitter)
                        actual_delay = max(0, actual_delay)  # Ensure non-negative delay
                        
                        print(f"Request failed with error: {e}. Retrying in {actual_delay:.2f} seconds...")
                        await asyncio.sleep(actual_delay)
                
                # Check if the model wants to use tools
                if stop_reason == "tool_use":
                    print("Model is requesting tool use")
                    
                    # Store the assistant's message with tool requests in our conversation
                    conversation.append({
                        "role": "assistant",
                        "content": content
                    })
                    
                    # Extract all tool use requests
                    tool_use_requests = []
                    for idx, item in enumerate(content):
                        if "toolUse" in item:
                            tool_use_requests.append(item["toolUse"])
                    
                    # Log the requests we've identified
                    print(f"Found {len(tool_use_requests)} tool requests")
                    for req in tool_use_requests:
                        print(f"  - {req.get('name')} (ID: {req.get('toolUseId')})")
                    
                    # Collect all tool results in the same order as they were requested
                    tool_results = []
                    
                    for req in tool_use_requests:
                        tool_name = req.get("name")
                        tool_input = req.get("input", {})
                        tool_id = req.get("toolUseId")
                        
                        print(f"Executing tool: {tool_name} with ID {tool_id}")
                        
                        # Apply exponential backoff for tool execution as well
                        tool_retry_count = 0
                        while tool_retry_count <= MAX_RETRIES:
                            try:
                                result = await self.execute_tool(tool_name, tool_input)
                                    # print("this is the current result", result)
                                
                                # Convert complex result objects to string
                                if isinstance(result, list) and len(result) > 0:
                                    if hasattr(result[0], 'text'):
                                        result = result[0].text

                                tool_results.append({
                                    "toolResult": {
                                        "toolUseId": tool_id,
                                        "content": [{"text": str(result)}]
                                    }
                                })
                                
                                # If tool execution succeeds, break the retry loop
                                break
                                
                            except Exception as e:
                                tool_retry_count += 1
                                if tool_retry_count > MAX_RETRIES:
                                    print(f"Max retries ({MAX_RETRIES}) exceeded for tool {tool_name}. Last error: {e}")
                                    # Add error result after max retries
                                    tool_results.append({
                                        "toolResult": {
                                            "toolUseId": tool_id,
                                            "content": [{"text": f"Error after {MAX_RETRIES} attempts: {str(e)}"}],
                                            "status": "error"
                                        }
                                    })
                                    break
                                
                                # Calculate delay with exponential backoff and jitter
                                delay = min(BASE_DELAY * (2 ** (tool_retry_count - 1)), MAX_DELAY)
                                # Add jitter (±20% randomness)
                                jitter = delay * 0.2
                                actual_delay = delay + random.uniform(-jitter, jitter)
                                actual_delay = max(0, actual_delay)  # Ensure non-negative delay
                                
                                print(f"Tool execution failed with error: {e}. Retrying in {actual_delay:.2f} seconds...")
                                await asyncio.sleep(actual_delay)
                    
                    # Add all tool results in a single message
                    conversation.append({
                        "role": "user",
                        "content": tool_results  # List of all tool results
                    })
                    
                    print(f"Added {len(tool_results)} tool results to conversation")
                    
                    # Continue loop to make another API call with updated conversation
                    
                else:
                    # Model has provided a final answer
                    print("Model has provided a final answer")
                    
                    # Extract text from content
                    text_parts = []
                    for item in content:
                        if isinstance(item, dict) and "text" in item:
                            text_parts.append(item["text"])
                    
                    result = "\n".join(text_parts)
                    return result if result else "No text content found in response"
                    
        except Exception as e:
            print(f"Error in process_query: {str(e)}")
            import traceback
            traceback.print_exc()
            raise

## System Prompt Overview

The system prompt for our Airbnb finder instructs Mistral Large to act as a personalized accommodation recommendation assistant:

In [15]:
SYSTEM_PROMPT = """
    You are an Airbnb recommendation assistant with access to the following MCP tools:

    Airbnb Tools:
    - airbnb_search: Search for Airbnb listings by location, dates, and filters
    - airbnb_listing_details: Get detailed information about a specific Airbnb listing
    
    Process for making recommendations:
    1. Use airbnb_search to find accommodations matching the user's criteria
    2. Use airbnb_listing_details to find listing details
    
    When recommending:
    - Consider the user's preferences for location, price range, amenities, and property type
    - Provide a diverse set of options when possible
    - Highlight key features of each property that match the user's preferences
    
    Format your response conversationally, explaining why each recommendation would suit the user.
    If you need any information, use the appropriate tool to get it.
"""

## Running the Airbnb Finder

Let's set up and run our Airbnb finder:

In [22]:
async def main():
    try:
        print("1. Initializing Airbnb finder...")
        finder = AirbnbFinder()
        
        print("2. Connecting to MCP servers...")
        await finder.connect_to_mcp_servers()
        
        print("3. Getting available tools...")
        tools = await finder.get_available_tools()
        print(f"Available tools: {tools}")
        
        # Example query
        location = "miami"
        query = "show me details about an accomodation in miami under 300usd for 4 adults from april 12 to april 15."
        
        print("4. Processing query...")
        response = await finder.process_query(query, location)
        print("5. Got response!")
        print(response)
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        
        raise e

await main()

1. Initializing Airbnb finder...
2. Connecting to MCP servers...
Connecting to airbnb server...
Successfully connected to airbnb server. Available tools:
  - airbnb_search: Search for Airbnb listings with various filters and pagination. Provide direct links to the user
  - airbnb_listing_details: Get detailed information about a specific Airbnb listing. Provide direct links to the user
3. Getting available tools...
Available tools: [{'name': 'airbnb_search', 'description': 'Search for Airbnb listings with various filters and pagination. Provide direct links to the user', 'input_schema': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'Location to search for (city, state, etc.)'}, 'placeId': {'type': 'string', 'description': 'Google Maps Place ID (overrides the location parameter)'}, 'checkin': {'type': 'string', 'description': 'Check-in date (YYYY-MM-DD)'}, 'checkout': {'type': 'string', 'description': 'Check-out date (YYYY-MM-DD)'}, 'adults': {'type': '